# dependencies
upload the Reacher-v4 agent to /content/ppo_Reacher-v4.zip

In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 29.3 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install seals
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.

# environment and expert

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
def make_env_expert_rollouts(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
        "Reacher-v4", # seals:seals/CartPole-v0
        rng=np.random.default_rng(SEED),
        n_envs=1,
        post_wrappers=[
            lambda env, _: RolloutInfoWrapper(env)
        ],  # needed for computing rollouts later
        parallel=True,
    )
    expert = load_policy(
        "ppo",
        path = "/content/ppo_Reacher-v4.zip", #-self-trained Reacher expert
        venv=env,
    )
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=50, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )

    return env, expert, rollouts


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
def make_PPO_learner_and_trainer(env, expert, rollouts):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    learner_PPO = PPO(
        env=env,
        policy=MlpPolicy,
        batch_size=128,
        #clip_range=0.3,
        #ent_coef= 0.0007566389899529574,
        learning_rate=1e-4, #1.943992487657563e-5
        gae_lambda=0.8,
        max_grad_norm=0.9,
        gamma=0.995,
        n_epochs=10,
        seed=SEED,
    )
    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
    )
    gail_trainer = GAIL(
        demonstrations = rollouts, # expert trajectories
        demo_batch_size = 50,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 8, #
        venv = env,
        gen_algo = learner_PPO,
        reward_net=reward_net,
        allow_variable_horizon=True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,

    )
    return learner_PPO, gail_trainer

In [8]:
def make_TRPO_learner_and_trainer(env, expert, rollouts):
    global SEED
    learner_TRPO = TRPO(
        env=env,
        policy = MlpPolicy,
        batch_size = 128,
        learning_rate = 1e-4, # 1.943992487657563e-5
        gae_lambda = 0.8,
        gamma = 0.995,
        n_critic_updates = 10,
        seed = SEED,
        target_kl = 0.01
    )

    reward_net = BasicRewardNet(
        observation_space=env.observation_space,
        action_space=env.action_space,
        normalize_input_layer=RunningNorm,
        #normalize_output_layer=RunningNorm,
    )

    gail_trainer_TRPO = GAIL(
        demonstrations = rollouts, # expert demo is here
        demo_batch_size = 50,
        gen_replay_buffer_capacity = 16384,
        n_disc_updates_per_round = 8, #
        venv = env,
        gen_algo = learner_TRPO,
        reward_net = reward_net,
        allow_variable_horizon = True,
        # gen_train_timesteps = 1000
        init_tensorboard=True,
        init_tensorboard_graph=True,
    )
    return learner_TRPO, gail_trainer_TRPO

# GAIL-PPO 1 trajectory

In [ ]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
learner_PPO_1, gail_trainer_PPO_1 = make_PPO_learner_and_trainer(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [ ]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_PPO_1.train(600000)
# An upper bound on the number of transitions to sample from the environment during training.

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_1, env_1, 20, return_episode_rewards=True
)

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -12.0647763 +/- 4.1092375105641334
Rewards after training: -12.958779900000001 +/- 4.487480748432074


# GAIL-TRPO 1 trajectory


use the same env, expert, rollout as previous

In [ ]:
env_1, expert_1, rollouts_1 = make_env_expert_rollouts(1)
learner_TRPO_1, gail_trainer_TRPO_1 = make_TRPO_learner_and_trainer(env_1, expert_1, rollouts_1)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards: -5.0036284 +/- 1.6795412761026864


In [ ]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_TRPO_1, env_1, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_TRPO_1.train(600000) #进度条显示的是gen_train_timestep除以这里的input

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_TRPO_1, env_1, 20, return_episode_rewards=True
)

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -12.0647763 +/- 4.1092375105641334
Rewards after training: -23.06562175 +/- 15.667347678037405


# GAIL-PPO 4 trajectory

In [ ]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_PPO_4, gail_trainer_PPO_4 = make_PPO_learner_and_trainer(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [ ]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_PPO_4.train(600000) # 1000000

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_4, env_4, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -8.3043686 +/- 3.211366761803942


# GAIL-TRPO 4 trajectory


use the same env, expert, rollout as previous

In [ ]:
env_4, expert_4, rollouts_4 = make_env_expert_rollouts(4)
learner_TRPO_4, gail_trainer_TRPO_4 = make_TRPO_learner_and_trainer(env_4, expert_4, rollouts_4)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards: -5.0036284 +/- 1.6795412761026864


In [ ]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_TRPO_4, env_4, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_TRPO_4.train(600000) #进度条显示的是gen_train_timestep除以这里的input

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_TRPO_4, env_4, 20, return_episode_rewards=True
)

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: -12.0647763 +/- 4.1092375105641334
Rewards after training: -7.987861 +/- 3.227449102354629


# GAIL-PPO 10 trajectory

In [ ]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_PPO_10, gail_trainer_PPO_10 = make_PPO_learner_and_trainer(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
print(rollouts_10.count)

<built-in method count of list object at 0x79b5259c6f40>


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [ ]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_PPO_10.train(600000) #进度条显示的是gen_train_timestep除以这里的input

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_PPO_10, env_10, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -5.594908000000001 +/- 1.4673345313272292


# GAIL-TRPO 10 trajectory

In [ ]:
env_10, expert_10, rollouts_10 = make_env_expert_rollouts(10)
learner_TRPO_10, gail_trainer_TRPO_10 = make_TRPO_learner_and_trainer(env_10, expert_10, rollouts_10)

Running with `allow_variable_horizon` set to True. Some algorithms are biased towards shorter or longer episodes, which may significantly confound results. Additionally, even unbiased algorithms can exploit the information leak from the termination condition, producing spuriously high performance. See https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html for more information.


In [ ]:
print(rollouts_10.count)

<built-in method count of list object at 0x79b524475980>


In [ ]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: -5.0036284 +/- 1.6795412761026864


In [ ]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner_TRPO_10, env_10, 20, return_episode_rewards=True
)

In [ ]:
gail_trainer_TRPO_10.train(600000) #进度条显示的是gen_train_timestep除以这里的input

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner_TRPO_10, env_10, 20, return_episode_rewards=True
)

In [ ]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: -8.46723935 +/- 3.0099406043928054
